In [1]:
import os
import boto3
from botocore.exceptions import ClientError, ReadTimeoutError
from botocore.config import Config

In [2]:
# session = boto3.Session(profile_name='ashu.joshi')
session = boto3.Session(profile_name='cdkuswest2')
config = Config(read_timeout=900)
brt = session.client(service_name='bedrock-runtime', config=config)
modelId = "meta.llama3-1-70b-instruct-v1:0"

In [11]:
def invokeModel(messageList):
    modelResponse = {}
    try:
        response = brt.converse(
                    modelId=modelId,
                    messages= messageList,
                    # system=[
                    #     { "text" : sysPrompt }
                    # ],
                    inferenceConfig={
                        # "maxTokens": 4096,
                        # "maxTokens": 2048,
                        "temperature": 0
                    },
                )
        response_message = response['output']['message']["content"][0]["text"]
        # print(response_message)
        inputTokens = response['usage']['inputTokens']
        outputTokens = response['usage']['outputTokens']
        totalTokens = response['usage']['totalTokens']
        latencyMs = response['metrics']['latencyMs']
        print(inputTokens, outputTokens, totalTokens, latencyMs)

        return response_message
        
    except (ClientError, ReadTimeoutError) as err:
        if 'Error' in err.response:
            modelResponse['errorCode'] = err.response['Error']['Code']
            modelResponse['errorMessage'] = err.response['Error']['Message']
        if 'ResponseMetadata' in err.response:
            modelResponse['HTTPStatusCode'] = err.response['ResponseMetadata']['HTTPStatusCode']
            modelResponse['RetryAttempts'] = err.response['ResponseMetadata']['RetryAttempts']

        return modelResponse


In [12]:
message_list = []
prompt = "Hello, How are you?"
summary_message = {
        "role": "user",
        "content": [
            { "text": prompt } 
        ],
}

message_list.append(summary_message)
response = invokeModel(message_list)
print(response)

20 53 73 1963


Hello! I'm just a language model, so I don't have feelings or emotions like humans do, but I'm functioning properly and ready to help with any questions or tasks you may have! How about you? How's your day going so far?


In [6]:
import yfinance as yf

def company_address(ticker: str)-> str:
    """
    Returns company address for input ticker.    
    e.g. company_address: AAPL
    Returns company address for ticker AAPL which is stock ticker for Apple Inc.
    """
    ticker_obj = yf.Ticker(ticker)
    info = ticker_obj.get_info()
    
    return " ".join([info[key] for key in ["address1", "city", "state", "zip", "country"]])

def fulltime_employees(ticker: str)-> int:
    """
    Returns fulltime employees count for input ticker.
    e.g. fulltime_employees: MSFT
    Returns fulltime employees count for ticker MSFT which is stock ticker for Microsoft.
    """
    ticker_obj = yf.Ticker(ticker)
    info = ticker_obj.get_info()
    
    return info["fullTimeEmployees"]

def last_close_price(ticker: str)-> float:
    """
    Returns last close price for input ticker.
    e.g. last_close_price: MSFT
    Returns latest close price for ticker MSFT which is stock ticker for Microsoft.
    """
    ticker_obj = yf.Ticker(ticker)
    info = ticker_obj.get_info()
    
    return info["previousClose"]

def EBITDA(ticker: str)-> float:
    """
    Returns EBITDA for input ticker.
    e.g. EBITDA: AAPL
    Returns EBITDA for ticker AAPL which is stock ticker for Apple.
    """
    ticker_obj = yf.Ticker(ticker)
    info = ticker_obj.get_info()
    
    return info["ebitda"]

def total_debt(ticker: str)-> float:
    """
    Returns total debt for input ticker.
    e.g. total_debt: AAPL
    Returns total debt for ticker AAPL which is stock ticker for Apple.
    """
    ticker_obj = yf.Ticker(ticker)
    info = ticker_obj.get_info()
    
    return info["totalDebt"]

def total_revenue(ticker: str)-> float:
    """
    Returns total revenue for input ticker.
    e.g. total_revenue: MSFT
    Returns total revenue for ticker MSFT which is stock ticker for Microsoft.
    """
    ticker_obj = yf.Ticker(ticker)
    info = ticker_obj.get_info()
    
    return info["totalRevenue"]

def debt_to_equity_ratio(ticker: str)-> float:
    """
    Returns debt to equity ratio for input ticker.
    e.g. debt_to_equity_ratio: AAPL
    Returns debt to equity ratio for ticker AAPL which is stock ticker for Apple.
    """
    ticker_obj = yf.Ticker(ticker)
    info = ticker_obj.get_info()
    
    return info["debtToEquity"]

tools = [company_address, fulltime_employees, last_close_price, EBITDA, total_debt, total_revenue, debt_to_equity_ratio]

In [7]:
tools_with_desc = "\n\n".join([tool.__name__ + tool.__doc__ for tool in tools])

print(tools_with_desc)

company_address
    Returns company address for input ticker.    
    e.g. company_address: AAPL
    Returns company address for ticker AAPL which is stock ticker for Apple Inc.
    

fulltime_employees
    Returns fulltime employees count for input ticker.
    e.g. fulltime_employees: MSFT
    Returns fulltime employees count for ticker MSFT which is stock ticker for Microsoft.
    

last_close_price
    Returns last close price for input ticker.
    e.g. last_close_price: MSFT
    Returns latest close price for ticker MSFT which is stock ticker for Microsoft.
    

EBITDA
    Returns EBITDA for input ticker.
    e.g. EBITDA: AAPL
    Returns EBITDA for ticker AAPL which is stock ticker for Apple.
    

total_debt
    Returns total debt for input ticker.
    e.g. total_debt: AAPL
    Returns total debt for ticker AAPL which is stock ticker for Apple.
    

total_revenue
    Returns total revenue for input ticker.
    e.g. total_revenue: MSFT
    Returns total revenue for ticker MSFT w

In [8]:
last_close_price("AAPL")

224.23

In [9]:
EBITDA("AAPL")

134660997120

In [13]:
system_prompt = f"""
You run in a loop of Thought, Action, PAUSE, Observation.
At the end of the loop you output an Answer
Use Thought to describe your thoughts about the question you have been asked.
Use Action to run one of the actions available to you - then return PAUSE.
Observation will be the result of running those actions.

Your available actions are:

{tools_with_desc}

Example session:

Question: What is last close price of Apple stock?
Thought: I should retrieve last close price using last_close_price
Action: last_close_price: AAPL
PAUSE

You will be called again with this:

Observation: 223.96

You then output:

Answer: The last close price of Apple stock is 223.96 USD.
""".strip()

In [29]:
class Agent:
    def __init__(self, system=""):
        self.system = system
        self.messages = [] ### Chat History
        # if self.system:
        #     self.messages.append({"role": "system", "content": system})

    def __call__(self, message):
        self.messages.append({"role": "user", "content": [ { "text": message } ]})
        result = self.execute()
        self.messages.append({"role": "assistant", "content": [{ "text" : result }]})
        return result

    # def execute(self):
    #     result = client.chat.completions.create(
    #         model="llama-3.1-70b-versatile", #llama3-70b-8192
    #         messages=self.messages,
    #         temperature=0
    #     )
    #     return result.choices[0].message.content


    # def invokeModel(self, message):
    def execute(self):
        modelResponse = {}
        # message_list = []
        # summary_message = {
        #     "role": "user",
        #     "content": [
        #         { "text": message } 
        #     ],
        # }

        # message_list.append(summary_message)
        try:
            response = brt.converse(
                        modelId=modelId,
                        messages= self.messages,
                        system=[
                            { "text" : self.system }
                        ],
                        inferenceConfig={
                            # "maxTokens": 4096,
                            # "maxTokens": 2048,
                            "temperature": 0
                        },
                    )
            response_message = response['output']['message']["content"][0]["text"]
            # print(response_message)
            inputTokens = response['usage']['inputTokens']
            outputTokens = response['usage']['outputTokens']
            totalTokens = response['usage']['totalTokens']
            latencyMs = response['metrics']['latencyMs']
            print(inputTokens, outputTokens, totalTokens, latencyMs)
    
            return response_message
            
        except (ClientError, ReadTimeoutError) as err:
            if 'Error' in err.response:
                modelResponse['errorCode'] = err.response['Error']['Code']
                modelResponse['errorMessage'] = err.response['Error']['Message']
            if 'ResponseMetadata' in err.response:
                modelResponse['HTTPStatusCode'] = err.response['ResponseMetadata']['HTTPStatusCode']
                modelResponse['RetryAttempts'] = err.response['ResponseMetadata']['RetryAttempts']
    
            return modelResponse


In [30]:
react = Agent(system_prompt)

In [32]:
# result = react.invokeModel("What is EBITDA of nvidia?")
result = react("What is EBITDA of nvidia?")
print(result)

463 26 489 1142


Thought: I should retrieve EBITDA using EBITDA
Action: EBITDA: NVDA
PAUSE


In [33]:
tool_result = EBITDA("NVDA")

tool_result

61184000000

In [34]:
next_prompt = "Observation: {}".format(tool_result)

next_prompt

'Observation: 61184000000'

In [36]:
# react.invokeModel(next_prompt)
result = react(next_prompt)
print(result)

542 21 563 965


Answer: The EBITDA of Nvidia is 61,184,000,000 USD.


In [37]:
result = react("Compare total revenue of Apple and Nvidia.")

print(result)

579 30 609 1222


Thought: I should retrieve total revenue using total_revenue for both Apple and Nvidia
Action: total_revenue: AAPL
PAUSE


In [38]:
tool_result = total_revenue("AAPL")
next_prompt = "Observation: {}".format(tool_result)
result = react(next_prompt)

print(result)

625 27 652 1144


Thought: I should retrieve total revenue using total_revenue for Nvidia
Action: total_revenue: NVDA
PAUSE


In [39]:
tool_result = total_revenue("NVDA")
next_prompt = "Observation: {}".format(tool_result)
result = react(next_prompt)

print(result)

668 50 718 1873


Answer: The total revenue of Apple is 391,034,994,688 USD, while the total revenue of Nvidia is 96,307,003,392 USD. Apple's total revenue is approximately 4 times that of Nvidia.


In [40]:
import re

action_re = re.compile('^Action: (\w+): (.*)$')   # python regular expression to selection action

def query(question, max_turns=5):
    i = 0
    react_agent = Agent(system_prompt)
    next_prompt = question
    while i < max_turns:
        i += 1
        result = react_agent(next_prompt)
        print(result)
        actions = [action_re.match(a) for a in result.split('\n') if action_re.match(a) ]
        
        if actions:
            action, action_input = actions[0].groups()
            if action not in known_actions:
                raise Exception("Unknown action: {}: {}".format(action, action_input))
            print("=> Running {} {}".format(action, action_input))
            observation = known_actions[action](action_input)
            print("Observation:", observation)
            next_prompt = "Observation: {}".format(observation)
        else:
            return

In [42]:
known_actions = dict([(tool.__name__, tool) for tool in tools])

known_actions

{'company_address': <function __main__.company_address(ticker: str) -> str>,
 'fulltime_employees': <function __main__.fulltime_employees(ticker: str) -> int>,
 'last_close_price': <function __main__.last_close_price(ticker: str) -> float>,
 'EBITDA': <function __main__.EBITDA(ticker: str) -> float>,
 'total_debt': <function __main__.total_debt(ticker: str) -> float>,
 'total_revenue': <function __main__.total_revenue(ticker: str) -> float>,
 'debt_to_equity_ratio': <function __main__.debt_to_equity_ratio(ticker: str) -> float>}

In [43]:
query("Compare total revenue of Apple and Nvidia.")


462 50 512 1896


Thought: To compare the total revenue of Apple and Nvidia, I need to retrieve the total revenue for each company using the total_revenue action.

Action: total_revenue: AAPL
PAUSE

(Waiting for the observation...)
=> Running total_revenue AAPL
Observation: 391034994688
528 50 578 1896


Thought: I have retrieved the total revenue for Apple, which is 391034994688. Now, I need to retrieve the total revenue for Nvidia using the total_revenue action.

Action: total_revenue: NVDA
PAUSE
=> Running total_revenue NVDA
Observation: 96307003392
594 76 670 2704


Thought: I have retrieved the total revenue for Nvidia, which is 96307003392. Now, I can compare the total revenue of Apple and Nvidia.

Answer: The total revenue of Apple is 391034994688, while the total revenue of Nvidia is 96307003392. Apple's total revenue is approximately 4.06 times that of Nvidia.


In [44]:
query("What day is it today")

459 136 595 4535


Thought: I need to determine the current date to answer this question, but I don't have a direct action to get the current date. However, I can use my available actions to get information about a company, but that won't help me determine the current date. I'll have to rely on my general knowledge to answer this question.

Action: None

PAUSE

Observation: None

Answer: I'm a large language model, I don't have real-time access to the current date. However, I can suggest ways for you to find out the current date, such as checking your device's clock or calendar, or searching online for the current date.


In [45]:
query("Compare total revenue of Apple and Amazon")

461 52 513 1918


Thought: I need to compare the total revenue of Apple and Amazon, so I should retrieve the total revenue for both companies using the total_revenue action.

Action: total_revenue: AAPL
PAUSE

(Waiting for the observation...)
=> Running total_revenue AAPL
Observation: 391034994688
529 40 569 1530


Thought: Now that I have the total revenue for Apple, I should retrieve the total revenue for Amazon using the total_revenue action.

Action: total_revenue: AMZN
PAUSE
=> Running total_revenue AMZN
Observation: 620127977472
585 55 640 1992


Thought: Now that I have the total revenue for both Apple and Amazon, I can compare them.

Answer: The total revenue of Amazon ($620,127,977,472) is higher than the total revenue of Apple ($391,034,994,688).
